In [5]:
#========================================================================
# Copyright 2019 Science Technology Facilities Council
# Copyright 2019 University of Manchester
#
# This work is part of the Core Imaging Library developed by Science Technology	
# Facilities Council and University of Manchester
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0.txt
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# 
#=========================================================================

## Introduction to reconstruction:
### FBP, CGLS

### Learning objectives
In this notebook you will learn how to set-up analytical (FBP) reconstruction and iterative reconstruction. You will learn how to set a number of iterations and check intermediate reconstruction results.

### CT reconstruction
There are two major classes of reconstruction algorithms: *analytic* and *iterative*. The most common analytic reconstruction algorithm is filtered back-projection (FBP). The FBP algorithm is derived from the Fourier Slice theorem which relates line integral measurements to two dimensional Fourier transform of an object’s slice. Although the Fourier Slice theorem provides straightforward solution for tomographic reconstruction, its practical implementation is challenging due to required interpolation from Polar to Cartesian coordinates in the Fourier space. In FBP-type reconstruction methods, projections are ﬁltered independently and then back-projected onto the plane of the tomographic slice. Filtration is used to compensate for nonuniform sampling of the Fourier space (higher frequencies have higher density of sampling points) by linear (Ramp) weighting of the frequency space.

As X-ray photons travel from an X-ray source to detector elements they interact with matter along their trajectories. In these interactions, photons are either absorbed or scattered, resulting in the attenuation of the incident X-ray. A quantitative description of the interaction of X-rays with matter is given by the Beer-Lambert law (or Beer’s law).
$$I^{l} = I^0 \mathrm{exp}\left( -\int_{l} f(g) \mathrm{d}l \right)$$
where $f(g)$ is the X-ray linear attenuation coefficient of the object at the position $g$ along a given linear X-ray trajectory $l$ from the source to the detector element. If $l$ is the entire trajectory from the source to the detector element, then $I^0$ corresponds to the X-ray intensity upon emission from the source and $I^{l}$ corresponds to the X-ray intensity upon incidence on the detector element. $I^{l}$ is typically called a transmission measurement, whereas a projection measurement is given by
$$G^{l} = -\log \left( \frac{I^{l}}{I^0} \right) = \int_{l} f(g) \mathrm{d}l$$

Ideally, $I^0$ is a single value, but real detector pixels do respond equally to photon flux. Secondly, pixels might have residual charge (so called dark current). Therefore, to convert $I^{l}$ to $G^{l}$, one needs to perform flat field correction. If $I^F$ is a flat field image (acquired with source on, without an object in the field of view) and $I^d$ is a dark field image (acquired with source off), then flat field correction is given by:
$$\frac{I-I^D}{I^F-I^D}$$

In [ ]:
# load flat and dark field images and wrap them as AcquisitionData objects
ag_f = ag.clone()
ag_f.angles = np.array([0], dtype = np.float32)
ag_d = ag_f.clone()
flat = ag_f.allocate()
#flat.fill(flat_tmp)
dark = ag_d.allocate()
#dark.fill(dark_tmp)

# and perform flat field correction and take negative logarithm
#sino = -log((ad - dark) / (flat - dark))

In [ ]:
# imports
from ccpi.framework import ImageData, AcquisitionData
from ccpi.framework import ImageGeometry, AcquisitionGeometry
from ccpi.astra.processors import FBP
from ccpi.framework import TestData
from ccpi.optimisation.algorithms import CGLS
from ccpi.astra.operators import AstraProjectorSimple

import os       
import tomophantom
from tomophantom import TomoP2D

import numpy as np
import matplotlib.pyplot as plt

Define some utilities for visualisation.

In [ ]:
def plot2D(datacontainer1, title1, datacontainer2, title2):
    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(15,15))
    plt.subplots_adjust(wspace = 0.5)

    ax1.set_title(title1)
    subplot1 = ax1.imshow(datacontainer1.as_array())
    plt.colorbar(subplot1, ax=ax1,fraction=0.0467, pad=0.02)
    
    ax2.set_title(title2)
    subplot2 = ax2.imshow(datacontainer2.as_array())
    plt.colorbar(subplot2, ax=ax2,fraction=0.0467, pad=0.02)
    
    plt.show()

Load data and set-up `AcquisitionGeometry`.

ImportError: No module named tomophantom

In [ ]:
# set-up AcquisitionGeometry   
N = 256 # set dimension of the phantom
n_angles = 180 # number of acquisition angles
angles = np.linspace(0, np.pi, n_angles, dtype=np.float32)
    
ag = AcquisitionGeometry(geom_type = 'parallel',
                         dimensions = '2D',
                         angles = angles, 
                         pixel_num_h = N,
                         pixel_size_h = 1)

# set-up ImageGeometry
ig = ImageGeometry(voxel_num_x  = N,
                   voxel_size_x = 1,
                   voxel_num_x = N,
                   voxel_size_y = 1)

In [ ]:
# Load  Shepp-Logan phantom 
model = 1 # select a model number from the library

path = os.path.dirname(tomophantom.__file__)
path_library2D = os.path.join(path, "Phantom2DLibrary.dat")
phantom_2D = TomoP2D.Model(model, N, path_library2D)
phantom_sino = TomoP2D.ModelSino(model, N, N, angles, path_library2D)
    
phantom = ig.allocate()
phantom.fill(phantom_2D)

sinogram = ag.allocate()
sinogram.fill(phantom_sino)

plot2D(sinogram, "sinogram", phantom, "phantom")

Iterative methods use an initial estimate of volume voxel values which is then iteratively updated to best reproduce acquired radiographic data. Since iterative methods involve forward- and back-projection steps, assumptions about data acquisition can be incorporated into the reconstruction procedure. However, iterative methods are computationally demanding, you will notice that it takes much longer to get reconstruction results with iterative methods.

$$\\underset{x}{\\mathrm{argmin}}\\begin{Vmatrix}A x - b\\end{Vmatrix}^2_2$$
where,
- $A$ is the projection operator
- $b$ is the acquired data
- $x$ is the solution

In [ ]:
#define the operator A
device = "gpu"
operator = AstraProjectorSimple(ig, ag, device)
    
#setup CGLS
x_init = ig.allocate()
cgls = CGLS(x_init=x_init, operator=operator, data=data)
cgls.max_iteration = 1000
cgls.update_objective_interval = 100

### Summary